In [2]:
import os
import string
import re
import pandas as pd
import requests
import numpy as np
from joblib import Parallel, delayed
from bs4 import BeautifulSoup as bs
from Fishy import fish_scraper

In [3]:
image_folder = "d://fish_scraper/"

In [4]:
wikiurl="https://en.wikipedia.org/wiki/List_of_freshwater_aquarium_fish_species"
table_class="sortable"
response=requests.get(wikiurl)
print(response.status_code)

200


In [5]:
soup = bs(response.text, 'html.parser')
fish_tables = soup.findAll('table',{'class':table_class}) #

In [6]:
all_tables = pd.read_html(str(fish_tables))

full_df = pd.DataFrame(columns=['Common name', 'Taxonomy'])

for table in all_tables:
    cols = table.columns.tolist()
    if 'Common name' not in cols:
        continue
    t = table[['Common name', 'Taxonomy']]
    full_df = full_df.append(t, ignore_index=True)

def clean_string(fish_str):
    pattern = r'[' + string.punctuation + ']'
    fish_str = re.sub(pattern, '', fish_str)
    fish_str.replace(" ", "-")
    return fish_str
    
    
full_df['Taxonomy'] = full_df['Taxonomy'].apply(lambda x: clean_string(x))

In [7]:
full_df['Taxonomy'].isna().any()

False

In [ ]:
if not os.path.exists('data'):
    os.makedirs('data')
    
full_df.to_csv('data/Fish_list.csv', index=False)

In [ ]:
def get_fish(fish, image_folder):
    f = fish_scraper(fish, image_folder, headless=False, num_images=200, max_res=(1400,1200), export_urls=True)
    f.run()

Parallel(n_jobs=8)(delayed(get_fish)(fish, image_folder) for fish in full_df['Taxonomy'])

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

headless = False

#fish = "Callichthys-callichthys"
webdriver_path="./webdriver/chromedriver.exe"
url = ("https://www.google.com/search?q=%s&source=lnms&tbm=isch&sa=X&ved=2ahUKEwie44_AnqLpAhUhBWMBHUFGD90Q_AUoAXoECBUQAw&biw=1920&bih=947"% (fish))
options = Options()
if headless:
    options.add_argument("--headless")
driver = webdriver.Chrome(webdriver_path, chrome_options=options)
driver.set_window_size(1400, 1050)
driver.get("https://www.google.com")

count = 0
missed_count = 0
driver.get(url)
time.sleep(3)
indx = 1
verbose = True
urls = []
fishes = list(full_df['Taxonomy'])

for fish in fishes[:5]:
    for i in range(20):
        imgurl = driver.find_element(By.XPATH, '//*[@id="islrg"]/div[1]/div[%s]/a[1]/div[1]/img' % (str(indx)),)
        imgurl.click()
        time.sleep(2)

        class_names = ["n3VNCb"] #["v4dQwb"] #
        images = [driver.find_elements(by=By.CLASS_NAME, value=class_name)
        for class_name in class_names if len(driver.find_elements(by=By.CLASS_NAME, value=class_name))!= 0][0]
                        # only download images that starts with http
        for image in images:
                            # only download images that starts with http
            src_link = image.get_attribute("src")
            if ("http" in src_link) and (not "encrypted" in src_link):
                if verbose:
                    print("[INFO] %d. %s" % (count, src_link))
                urls.append(src_link)
                count += 1
                break

        if count % 3 == 0:
            driver.execute_script("window.scrollTo(0, " + str(indx * 60) + ");")
        element = driver.find_element(by=By.CLASS_NAME, value="lxa62b.MIdC8d.jwwPNd") # mye4qd
        element.click()
        print("[INFO] Loading more photos")
        time.sleep(3)
self.driver.quit()

In [ ]:
from bs4 import BeautifulSoup as bs
import re

#fish = "Callichthys-callichthys"

for fish in full_df['Taxonomy']:
    html_page = requests.get(f"http://fishbase.de/summary/{fish}")
    soup = bs(html_page.content, "html.parser")

    txt = soup.find(title="English")
    if txt == None:
        continue
    fish_id = re.search("[0-9]+", str(txt))[0]
    print(f"{fish} - {fish_id}")

In [ ]:
txt

In [ ]:
full_df['Taxonomy'].values

In [ ]:
import json
with open("d:/fish_scraper/2/fish_img_urls.json") as f:
    url_dict = json.load(f)

In [ ]:
url_dict

In [ ]:
txt = soup.find(class_='slabel8') #
txt = soup.find_all('a')
txt

In [ ]:
txt

In [ ]:
soup

In [8]:
full_df.shape

(597, 2)